# 08: Procesamiento de Boletines

**Propósito:** Este *notebook* consolida todos los archivos `boletines.csv` (crudos, por período) de `data/01_raw/` en un único *dataset* maestro.

**Proceso:**
1.  **Carga y Consolidación:** Llama a `load_all_bulletin_files` para cargar y **deduplicar** todos los boletines por `boletin_id`.
2.  **Procesamiento LLM:** Inicializa un cliente de Ollama y llama a la función `extraer_materia_llm` (de `src/processing_utils.py`) sobre la columna `materias_str`.
3.  **Extracción de Features:** Desempaca la respuesta JSON del LLM (ej. `ambitos_detectados`).
4.  **Guardado:** Guarda el archivo maestro (`boletines_master_clean.parquet`) en `data/02_processed/`.

**Advertencia de Memoria:** Este *notebook* carga el **texto completo** de todos los proyectos de ley en la RAM. El archivo de salida será de **varios Gigabytes**.

**Dependencias:**
* `data/01_raw/[periodo]/boletines.csv` (Múltiples archivos)
* `src/processing_utils.py` (Debe contener `load_all_bulletin_files` y `extraer_materia_llm`)

**Salidas (Artifacts):**
* `data/02_processed/boletines_master_clean.parquet` (Archivo MUY grande)

In [1]:
import pandas as pd
from pathlib import Path
import sys
import logging
from tqdm.notebook import tqdm
from ollama import Client # <-- Importante
import json

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    from src.processing_utils import (
        load_all_bulletin_files, 
        extraer_materia_llm  
    )
except ImportError as e:
    logging.error(f"ERROR: No se pudieron importar las funciones desde /src. {e}")
    raise

# Registrar 'tqdm' con pandas
tqdm.pandas()

In [2]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"
DATA_DIR_PROCESSED = ROOT / "data" / "02_processed"

DATA_DIR_PROCESSED.mkdir(parents=True, exist_ok=True)

# Archivo de salida
OUTPUT_FILE = DATA_DIR_PROCESSED / "boletines_master_clean.parquet"

# --- Constantes del Modelo ---
OLLAMA_MODEL_NAME = 'llama3:instruct' # O el modelo que estés usando

logging.info(f"Archivo de Salida: {OUTPUT_FILE}")
logging.info(f"Usando modelo LLM: {OLLAMA_MODEL_NAME}")

2025-10-30 08:12:20,933 - INFO - Archivo de Salida: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed\boletines_master_clean.parquet
2025-10-30 08:12:20,937 - INFO - Usando modelo LLM: llama3:instruct


## 1. Conectar con el Cliente LLM (Ollama)

Inicializamos el cliente de Ollama. Asegúrese de que Ollama esté en ejecución.

In [7]:
try:
    ollama_client = Client(host='http://127.0.0.1:11434')
    # (Opcional) Especificar host si no es local:
    # ollama_client = Client(host='http://mi-servidor-ollama:11434')
    
    ollama_client.list() # Testear la conexión
    logging.info("Conexión con Ollama establecida exitosamente.")
except Exception as e:
    logging.error("ERROR FATAL: No se pudo conectar a Ollama.")
    logging.error("Asegúrese de que el servicio de Ollama esté en ejecución.")
    logging.error(f"Error: {e}")
    # raise e # Detener el notebook si la conexión falla

2025-10-30 08:15:44,420 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
2025-10-30 08:15:44,423 - INFO - Conexión con Ollama establecida exitosamente.


## 2. Carga, Consolidación y Deduplicación

Cargamos todos los `boletines.csv` y los unificamos en un solo DataFrame. La función `load_all_bulletin_files` se encarga de la deduplicación por `boletin_id`.

In [8]:
df_full = load_all_bulletin_files(DATA_DIR_RAW)

if not df_full.empty:
    display(df_full.head())
    print(f"Dimensiones del DataFrame consolidado y deduplicado: {df_full.shape}")
else:
    logging.error("No se cargaron datos. Deteniendo el notebook.")

2025-10-30 08:15:45,554 - INFO - Buscando archivos 'boletines.csv'...
2025-10-30 08:15:45,565 - INFO - Encontrados 6 archivos. Cargando...
2025-10-30 08:15:45,639 - INFO - DataFrame consolidado creado con 3528 filas.
2025-10-30 08:15:45,639 - INFO - Deduplicando por 'boletin_id'...
2025-10-30 08:15:45,642 - INFO - DataFrame deduplicado tiene 3286 boletines únicos.


,boletin_id,titulo,fecha_ingreso,iniciativa,camara_origen,etapa,leynro,link_mensaje_mocion,autores_json,materias_str,materias_json,boletin_id_consultado
0,2625-07,Moderniza la normativa reguladora de los arren...,15/11/2000,Moción,Senado,Tramitación terminada,Ley Nº 19.866,http://www.senado.cl/appsenado/index.php?mo=tr...,"[""R\u00edos Santander, Mario""]",PREDIOS URBANOS,"[""PREDIOS URBANOS""]",2625
1,3107-05,Crea una bonificación a la contratación de man...,17/10/2002,Mensaje,C.Diputados,Tramitación terminada,Ley Nº 19.853,http://www.senado.cl/appsenado/index.php?mo=tr...,[],BONIFICACION POR CONTRATACIÓN DE MANO DE OBRA,"[""BONIFICACION POR CONTRATACI\u00d3N DE MANO D...",3107
2,2727-11,Sobre los derechos y deberes de las personas e...,12/06/2001,Mensaje,C.Diputados,Archivado,NaN,http://www.senado.cl/appsenado/index.php?mo=tr...,[],SALIDA DE TROPAS,"[""SALIDA DE TROPAS""]",2727
3,3039-07,Reforma constitucional que establece la obliga...,03/09/2002,Mensaje,C.Diputados,Tramitación terminada,Ley Nº 19.876,http://www.senado.cl/appsenado/index.php?mo=tr...,[],EDUCACIÓN; REFORMA CONSTITUCIONAL,"[""EDUCACI\u00d3N"", ""REFORMA CONSTITUCIONAL""]",3039
4,3147-10,Aprueba el acuerdo por el que se establece una...,03/12/2002,Mensaje,C.Diputados,Tramitación terminada,D.S. Nº28,http://www.senado.cl/appsenado/index.php?mo=tr...,[],"ACUERDO DE ASOCIACIÓN CHILE-UNIÓN EUROPEA, SUS...","[""ACUERDO DE ASOCIACI\u00d3N CHILE-UNI\u00d3N ...",3147


Dimensiones del DataFrame consolidado y deduplicado: (3286, 12)


## 3. Procesamiento LLM: Clasificación de Materias

Este es el paso más lento. Aplicamos la función `extraer_materia_llm` a cada fila.
La función usa el `SYSTEM_PROMPT` que definimos para clasificar el texto de `materias_str`.

In [9]:
if not df_full.empty:
    logging.info("Iniciando clasificación de materias con LLM...")
    
    # Asegurarse de que la columna de materias exista y sea string
    if 'materias_str' not in df_full.columns:
        logging.error("No se encontró la columna 'materias_str'. Saltando paso de LLM.")
        df_full['llm_output'] = None
    else:
        # Rellenar NaNs con string vacío para que .progress_apply no falle
        df_full['materias_str'] = df_full['materias_str'].fillna("")
        
        # Aplicar la función de LLM
        # (Esto llamará a Ollama por cada fila)
        tqdm.pandas(desc="Clasificando Materias (LLM)")
        df_full['llm_output'] = df_full['materias_str'].progress_apply(
            lambda x: extraer_materia_llm(
                client=ollama_client, 
                materias_str=x, 
                model_name=OLLAMA_MODEL_NAME
            )
        )
    
    logging.info("Clasificación de materias finalizada.")
    display(df_full[['materias_str', 'llm_output']].sample(5))
else:
    logging.warning("DataFrame vacío, saltando procesamiento.")

2025-10-30 08:15:48,833 - INFO - Iniciando clasificación de materias con LLM...


Clasificando Materias (LLM):   0%|          | 0/3286 [00:00<?, ?it/s]

2025-10-30 08:15:55,207 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:16:05,527 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:16:12,563 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:16:20,123 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:16:32,917 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:16:32,917 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 08:16:47,098 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:16:54,181 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:16:59,891 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:17:09,016 - INFO - HTTP Request: POST http://127.0.0.1:11434

2025-10-30 08:26:35,248 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:26:45,105 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:26:50,708 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:26:59,417 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:27:08,180 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:27:13,281 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:27:21,352 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:27:32,457 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:27:39,439 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:27:46,724 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/cha

2025-10-30 08:37:08,793 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:37:17,900 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:37:23,450 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:37:33,392 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:37:42,325 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:37:49,919 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:37:58,002 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:38:04,899 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 08:38:04,900 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 08:38:11,643 - INFO - HTTP Request: POST http://127.0.0.1:11434

...

2025-10-30 16:05:57,257 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:05:57,257 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:05:57,258 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:05:57,259 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:05:57,259 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:05:57,261 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:06:16,894 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 16:06:31,125 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 16:06:48,878 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 16:06:48,879 - INFO - e

2025-10-30 16:11:57,436 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 16:12:14,327 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 16:12:14,328 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:12:26,096 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 16:12:32,673 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 16:12:32,674 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:12:45,472 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 16:12:45,474 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:12:45,474 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:13:09,365 - INFO - HTTP Request: POS

2025-10-30 16:15:35,883 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:15:35,883 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:15:35,884 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:15:35,884 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:15:35,884 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:15:35,885 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:15:35,885 - INFO - extraer_materia_llm: Texto de materias omitido (demasiado corto o inválido).
2025-10-30 16:15:46,276 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 16:15:57,227 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-10-30 16:15:57,228 - 

2025-10-30 16:16:54,727 - INFO - Clasificación de materias finalizada.


,materias_str,llm_output
1119,ESTABLECIMIENTOS EDUCACIONALES; SUBVENCIONES,{'materias_originales': 'ESTABLECIMIENTOS EDUC...
2139,ACCIDENTES DE TRABAJO; ENFERMEDAD PROFESIONAL;...,{'materias_originales': 'ACCIDENTES DE TRABAJO...
1490,INVERSIÓN PRIVADA; OBRAS DE RIEGO Y DRENAJE,{'materias_originales': 'INVERSIÓN PRIVADA; OB...
540,EXONERADOS POLITICOS,{'materias_originales': 'EXONERADOS POLITICOS'...
129,DELITOS SEXUALES; MENORES,{'materias_originales': 'DELITOS SEXUALES; MEN...


## 4. Desempaquetar, Limpiar y Guardar

Desempaquetamos la respuesta JSON del LLM, seleccionamos las columnas finales y guardamos el archivo Parquet.

In [10]:
if not df_full.empty:
    # --- 1. Desempaquetar JSON del LLM ---
    logging.info("Desempaquetando resultados del LLM...")
    
    # (Usar .get() para evitar errores si 'llm_output' es None o el dict no tiene la llave)
    df_full['ambitos'] = df_full['llm_output'].apply(
        lambda x: x.get('ambitos_detectados') if isinstance(x, dict) else ['no cumple']
    )
    
    # Convertir la lista de ámbitos a un string JSON para guardarla
    df_full['ambitos_json'] = df_full['ambitos'].apply(
        lambda x: json.dumps(x) if isinstance(x, list) else json.dumps(['no cumple'])
    )

    # --- 2. Seleccionar y Renombrar Columnas ---
    logging.info("Seleccionando y renombrando columnas finales...")
    
    cols_to_keep = {
        'boletin_id': 'boletin_id',
        'titulo': 'titulo',
        'fecha_ingreso': 'fecha_ingreso',
        'iniciativa': 'iniciativa',
        'camara_origen': 'camara_origen',
        'etapa': 'etapa',
        'autores_json': 'autores_json',
        'materias_str': 'materias_raw',
        'ambitos_json': 'ambitos_json'
    }
    
    columnas_validas = [col for col in cols_to_keep.keys() if col in df_full.columns]
    df_final = df_full[columnas_validas]
    df_final = df_final.rename(columns=cols_to_keep)
    
    # --- 3. Guardar ---
    logging.info(f"Guardando archivo Parquet en {OUTPUT_FILE}...")
    try:
        df_final.to_parquet(OUTPUT_FILE, index=False)
        logging.info(f"Guardado exitosamente: {OUTPUT_FILE}")
        logging.info(f"Dimensiones del DataFrame maestro: {df_final.shape}")
    except Exception as e:
        logging.error(f"ERROR al guardar en Parquet: {e}")

    display(df_final.head())
else:
    logging.error("DataFrame procesado vacío. No se guardó ningún archivo.")

logging.info("--- Procesamiento de Boletines finalizado ---")

2025-10-30 17:02:08,897 - INFO - Desempaquetando resultados del LLM...
2025-10-30 17:02:08,912 - INFO - Seleccionando y renombrando columnas finales...
2025-10-30 17:02:08,919 - INFO - Guardando archivo Parquet en C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed\boletines_master_clean.parquet...
2025-10-30 17:02:09,064 - INFO - Guardado exitosamente: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed\boletines_master_clean.parquet
2025-10-30 17:02:09,065 - INFO - Dimensiones del DataFrame maestro: (3286, 9)


,boletin_id,titulo,fecha_ingreso,iniciativa,camara_origen,etapa,autores_json,materias_raw,ambitos_json
0,2625-07,Moderniza la normativa reguladora de los arren...,15/11/2000,Moción,Senado,Tramitación terminada,"[""R\u00edos Santander, Mario""]",PREDIOS URBANOS,"[""Vivienda y Urbanismo""]"
1,3107-05,Crea una bonificación a la contratación de man...,17/10/2002,Mensaje,C.Diputados,Tramitación terminada,[],BONIFICACION POR CONTRATACIÓN DE MANO DE OBRA,"[""Trabajo y Previsi\u00f3n"", ""Econom\u00eda y ..."
2,2727-11,Sobre los derechos y deberes de las personas e...,12/06/2001,Mensaje,C.Diputados,Archivado,[],SALIDA DE TROPAS,"[""Seguridad y Justicia"", ""Gobierno y Pol\u00ed..."
3,3039-07,Reforma constitucional que establece la obliga...,03/09/2002,Mensaje,C.Diputados,Tramitación terminada,[],EDUCACIÓN; REFORMA CONSTITUCIONAL,"[""Educaci\u00f3n"", ""Gobierno y Pol\u00edtica""]"
4,3147-10,Aprueba el acuerdo por el que se establece una...,03/12/2002,Mensaje,C.Diputados,Tramitación terminada,[],"ACUERDO DE ASOCIACIÓN CHILE-UNIÓN EUROPEA, SUS...","[""Relaciones Exteriores"", ""no cumple""]"


2025-10-30 17:02:09,071 - INFO - --- Procesamiento de Boletines finalizado ---


In [14]:
df_final['ambitos_json'].unique()

261